# Converting .tsv files to .csv 

In [1]:
import pandas as pd

In [2]:
# File paths
input_file = #Add your file path for the title.ratings.tsv

# Read the TSV data into a DataFrame
ratings_df = pd.read_csv(input_file, sep='\t', low_memory=False)  # Read as TSV

In [3]:
# File paths
input_file = #Add your file path for the title.basics.tsv

# Read the TSV file, replace '\N' with NaN, and handle bad lines
basics_df = pd.read_csv(input_file, sep='\t', na_values=['\\N'], low_memory=False)


In [4]:
# Filter out adult titles
basics_df1 = basics_df.loc[basics_df['isAdult'] == 0]

In [5]:
# Remove isAdult column which is now 0 for every row
del basics_df1['isAdult']

In [6]:
# Reviewing remaining columns and datatypes
basics_df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10789440 entries, 0 to 11147720
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   startYear       float64
 5   endYear         float64
 6   runtimeMinutes  object 
 7   genres          object 
dtypes: float64(2), object(6)
memory usage: 740.9+ MB


In [7]:
# Filter for only movies made in 1939 or after
basics_df2 = basics_df1.loc[basics_df1['startYear'] >= 1939]

In [8]:
# Change startYear as type int
basics_df2 = basics_df2.astype({'startYear':int}, errors='raise')

In [9]:
# Filter to only include movies
basics_df3 = basics_df2.loc[basics_df2['titleType'] == 'movie']

In [10]:
# Delete titleType as all remaining rows are movies
del basics_df3['titleType']

In [11]:
# Delete endYear as this column does not apply to movies
del basics_df3['endYear']

In [12]:
# Remove all rows with null values
basics_df4 = basics_df3.dropna(how = 'any')

In [13]:
# Delete originalTitle column as it is less useful than primaryTitle
del basics_df4['originalTitle']

In [14]:
# Convert runtimeMinutes to integer
basics_df5 = basics_df4.astype({'runtimeMinutes':int}, errors='raise')

In [15]:
# Overview of cleaned data
basics_df5.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres
13077,tt0013274,Istoriya grazhdanskoy voyny,2021,94,Documentary
15480,tt0015724,Dama de noche,1993,102,"Drama,Mystery,Romance"
18588,tt0018867,Escape from Hong Kong,1942,60,"Adventure,Mystery,War"
21267,tt0021617,Arizona Territory,1950,56,Western
21704,tt0022064,Lebbra bianca,1951,80,Drama


In [16]:
# Create genres array with each unique genre in genres column
genres = basics_df5['genres'].str.split(',').explode().unique()
genres

array(['Documentary', 'Drama', 'Mystery', 'Romance', 'Adventure', 'War',
       'Western', 'Musical', 'Comedy', 'Thriller', 'Crime', 'Film-Noir',
       'History', 'Biography', 'Fantasy', 'Action', 'Sport', 'Family',
       'Music', 'Horror', 'Animation', 'Sci-Fi', 'News', 'Talk-Show',
       'Reality-TV', 'Game-Show'], dtype=object)

In [17]:
# Create Boolean genre columns for one-hot encoding for easier data manipulation
for genre in genres:
    basics_df5[genre] = basics_df5['genres'].str.contains(genre).astype(int)

In [18]:
# Delete genres column as it is now redundant
del basics_df5['genres']

In [19]:
# Renaming genre columns to remove '-'
basics_df6 = basics_df5.rename(columns={'Film-Noir':'FilmNoir','Sci-Fi':'SciFi','Talk-Show':'TalkShow','Reality-TV':'RealityTV','Game-Show':'GameShow'})

In [20]:
# Remove single quotes from titles that interfere with uploading
def remove_single_quotes(text):
    if isinstance(text, str):
        return text.replace("'", "")  # Remove single quotes
    return text
# Apply the function to remove single quotes
basics_df6['primaryTitle'] = basics_df6['primaryTitle'].apply(remove_single_quotes)

In [21]:
# Filtering for rows that have IDs that correspond with each other in both dataframes
ratings_df1 = ratings_df[ratings_df['tconst'].isin(basics_df6['tconst'])]

In [22]:
basics_df7 = basics_df6[basics_df6['tconst'].isin(ratings_df1['tconst'])]

In [23]:
basics_df7.head()

,tconst,primaryTitle,startYear,runtimeMinutes,Documentary,Drama,Mystery,Romance,Adventure,War,...,Sport,Family,Music,Horror,Animation,SciFi,News,TalkShow,RealityTV,GameShow
13077,tt0013274,Istoriya grazhdanskoy voyny,2021,94,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15480,tt0015724,Dama de noche,1993,102,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
18588,tt0018867,Escape from Hong Kong,1942,60,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
21267,tt0021617,Arizona Territory,1950,56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21704,tt0022064,Lebbra bianca,1951,80,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
output_file = 'Resources/CSV/title.basics_clean.csv'

# Save the processed DataFrame to a new CSV file
basics_df7.to_csv(output_file, index=False)

In [25]:
output_file = 'Resources/CSV/title.ratings_clean.csv'

# Save the processed DataFrame to a new CSV file
ratings_df1.to_csv(output_file, index=False)

# Enter data into postgres using the table schema found in the repo 

In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
from config import engine_key

engine = create_engine(f'postgresql+psycopg2://{engine_key}')
conn = engine.connect()


In [3]:
# Query All Records in the the Database
title_basics = pd.read_sql("SELECT * FROM title_basics", conn)

In [4]:
title_basics

,tconst,primarytitle,startyear,runtimeminutes,documentary,drama,mystery,romance,adventure,war,...,family,music,horror,animation,scifi,news,talkshow,realitytv,gameshow,adult
0,tt0013274,Istoriya grazhdanskoy voyny,2021,94,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,tt0015724,Dama de noche,1993,102,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tt0018867,Escape from Hong Kong,1942,60,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
3,tt0021617,Arizona Territory,1950,56,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,tt0022064,Lebbra bianca,1951,80,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263370,tt9916190,Safeguard,2020,95,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
263371,tt9916270,Il talento del calabrone,2020,84,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
263372,tt9916362,Coven,2020,92,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
263373,tt9916538,Kuambil Lagi Hatiku,2019,123,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
# Query All Records in the the Database
title_ratings = pd.read_sql("SELECT * FROM title_ratings", conn)

In [6]:
# Merge ratings and basics dataframes
imdb_df = pd.merge(title_basics,title_ratings,on='tconst')

In [7]:
# Reordering columns 
new_imdb_df = imdb_df[['tconst', 'primarytitle', 'startyear', 'runtimeminutes','averagerating','numvotes', 'documentary',
       'drama', 'mystery', 'romance', 'adventure', 'war', 'western', 'musical',
       'comedy', 'thriller', 'crime', 'filmnoir', 'history', 'biography',
       'fantasy', 'action', 'sport', 'family', 'music', 'horror', 'animation',
       'scifi', 'news', 'talkshow', 'realitytv', 'gameshow']]

In [9]:
#Change 'startyear' column to 'year' and 'primarytitle' to 'title'
new_imdb_df = new_imdb_df.rename(columns={'startyear': 'year', 'primarytitle': 'title'})

In [10]:
new_imdb_df.head()

,tconst,title,year,runtimeminutes,averagerating,numvotes,documentary,drama,mystery,romance,...,sport,family,music,horror,animation,scifi,news,talkshow,realitytv,gameshow
0,tt0013274,Istoriya grazhdanskoy voyny,2021,94,6.7,74,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,tt0015724,Dama de noche,1993,102,6.3,31,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,tt0018867,Escape from Hong Kong,1942,60,5.6,46,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,tt0021617,Arizona Territory,1950,56,6.1,62,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,tt0022064,Lebbra bianca,1951,80,5.2,62,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
output_file = 'Resources/CSV/imdb_merged.csv'

# Save the processed DataFrame to a new CSV file
new_imdb_df.to_csv(output_file, index=False)